In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected")

CUDA available: True
GPU Name: NVIDIA A100-SXM4-40GB


In [20]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("Is CUDA available:", torch.cuda.is_available())

PyTorch version: 2.4.1
CUDA version: 11.8
Is CUDA available: True


In [21]:
import os
import sys
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import argparse
import torch.optim
import torch.backends.cudnn as cudnn
import torch
import torch.optim as optim
import random
import torch.nn as nn
from torch.utils.data import Subset, DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import faiss
from scipy.sparse import csr_matrix
import time
import math
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.cluster import normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import multiprocessing
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.profiler import profile, record_function, ProfilerActivity
from torch.cuda.amp import autocast

In [23]:
# Define custom dataset class for Xray images
class ChestXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("L")
        if self.transform:
            img = self.transform(img)
        return img

# Initial transform for grayscale images
initial_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset for grayscale images
dataset_path = "/gpfs/workdir/islamm/rotated_datasets_without_NoFindings" 
dataset = ChestXrayDataset(root_dir=dataset_path, transform=initial_transform)
dataloader = DataLoader(dataset, batch_size=512, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

print("Dataset loaded with {} images.".format(len(dataset)))

Dataset loaded with 310554 images.


In [24]:

# AlexNet model definition
__all__ = [ 'AlexNet', 'alexnet']
 
# (number of filters, kernel size, stride, pad)
CFG = {
    '2012': [(96, 11, 4, 2), 'M', (256, 5, 1, 2), 'M', (384, 3, 1, 1), (384, 3, 1, 1), (256, 3, 1, 1), 'M']
}

class AlexNet(nn.Module):
    def __init__(self, features, num_classes, sobel):
        super(AlexNet, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(nn.Dropout(0.5),
                            nn.Linear(256 * 6 * 6, 4096),
                            nn.ReLU(inplace=True),
                            nn.Dropout(0.5),
                            nn.Linear(4096, 4096),
                            nn.ReLU(inplace=True))

        self.top_layer = nn.Linear(4096, num_classes)
        self._initialize_weights()

        if sobel:
            sobel_filter = nn.Conv2d(1, 2, kernel_size=3, stride=1, padding=1) 
            sobel_filter.weight.data[0, 0].copy_(
                torch.FloatTensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
            )
            sobel_filter.weight.data[1, 0].copy_(
                torch.FloatTensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
            )
            sobel_filter.bias.data.zero_()
            self.sobel = sobel_filter
            for p in self.sobel.parameters():
                p.requires_grad = False
        else:
            self.sobel = None

    def forward(self, x, return_features=True):
        if self.sobel:
            x = self.sobel(x)
        x = self.features(x)
        if return_features:
            return x.view(x.size(0), -1) 
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        if self.top_layer:
            x = self.top_layer(x)
        return x


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def make_layers_features(cfg, input_dim, bn):
    layers = []
    in_channels = input_dim
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=3, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v[0], kernel_size=v[1], stride=v[2], padding=v[3])
            if bn:
                layers += [conv2d, nn.BatchNorm2d(v[0]), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v[0]
    return nn.Sequential(*layers)

def alexnet(sobel=True, bn=True, out=1000):
    dim = 2 + int(not sobel)
    model = AlexNet(make_layers_features(CFG['2012'], dim, bn=bn), out, sobel)
    return model

In [25]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_features(dataloader, model, N, device):
    """
    Compute features for the entire dataset and store in a pre-allocated numpy array.
    Args:
        dataloader (DataLoader): DataLoader for the dataset.
        model (nn.Module): CNN model used to extract features.
        N (int): Total number of samples in the dataset.
        device (torch.device): GPU or CPU device.
    Returns:
        np.ndarray: Numpy array of extracted features.
    """
    print("Computing features...")
    model.eval()
    batch_time = AverageMeter()
    end = time.time()

    # Pre-allocate numpy array for features
    for i, input_tensor in enumerate(tqdm(dataloader, desc="Feature Extraction")):
        input_tensor = input_tensor.to(device)
        with torch.no_grad():
            aux = model(input_tensor).cpu().numpy()

        # Initialize feature matrix on the first batch
        if i == 0:
            features = np.zeros((N, aux.shape[1]), dtype='float32')

        # Save extracted features
        if i < len(dataloader) - 1:
            features[i * dataloader.batch_size: (i + 1) * dataloader.batch_size] = aux
        else:
            features[i * dataloader.batch_size:] = aux

        # Measure batch time
        batch_time.update(time.time() - end)
        end = time.time()

    print(f"Feature extraction completed. Total time: {batch_time.sum:.2f} seconds")
    return features

In [26]:
def preprocess_features(features, pca_dim):
    """
    Preprocess features using PCA, whitening, and L2 normalization.
    Args:
        features (torch.Tensor): Raw features extracted from the model.
        pca_dim (int): Target dimensionality after PCA.
    Returns:
        np.ndarray: Preprocessed features ready for clustering.
    """
    features = features.astype('float32')

    # Apply PCA and whitening with Faiss
    ndim = features.shape[1]
    pca = faiss.PCAMatrix(ndim, pca_dim, eigen_power=-0.5)
    pca.train(features)
    features = pca.apply_py(features)

    # L2 normalization
    features /= np.linalg.norm(features, axis=1, keepdims=True)
    return features


In [27]:
def run_kmeans(features, num_clusters, verbose=False):
    """
    Perform k-means clustering on preprocessed features using FAISS GPU.
    Args:
        features (np.ndarray): Preprocessed feature vectors.
        num_clusters (int): Number of clusters.
        verbose (bool): If True, print k-means loss evolution.
    Returns:
        list: Cluster assignments for each feature vector.
        float: Final k-means loss.
    """
    n_data, d = features.shape

    # Configure k-means clustering
    clus = faiss.Clustering(d, num_clusters)
    clus.seed = np.random.randint(1234)
    clus.niter = 20
    clus.max_points_per_centroid = 10000000

    # Set GPU resources and configuration
    res = faiss.StandardGpuResources()
    flat_config = faiss.GpuIndexFlatConfig()
    flat_config.useFloat16 = True  # Enable float16 for speed
    flat_config.device = 0  # Set to GPU 0

    # Create the FAISS index
    index = faiss.GpuIndexFlatL2(res, d, flat_config)

    # Perform clustering
    clus.train(features, index)
    _, I = index.search(features, 1)  # Get cluster assignments
    losses = faiss.vector_to_array(clus.obj)  # K-means loss evolution

    if verbose:
        print(f"k-means loss evolution: {losses}")

    return I.flatten().tolist(), losses[-1]


In [28]:
# ReassignedDataset class
class ReassignedDataset(Dataset):
    """
    Dataset with images assigned pseudo-labels from clustering.
    Args:
        image_paths (list): List of image file paths.
        pseudo_labels (list): Cluster assignments (pseudo-labels).
        transform (callable, optional): Transformation pipeline for images.
    """
    def __init__(self, image_paths, pseudo_labels, transform=None):
        self.image_paths = image_paths
        self.pseudo_labels = pseudo_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        pseudo_label = self.pseudo_labels[idx]
        img = Image.open(img_path).convert("L") 
        if self.transform:
            img = self.transform(img)
        return img, pseudo_label

# Modify the cluster_assign function to use the correct arguments
def cluster_assign(images_lists, dataset, transform=None):
    """Creates a dataset from clustering, with clusters as labels.
    Args:
        images_lists (list of list): For each cluster, the list of image indexes
                                     belonging to this cluster.
        dataset (Dataset): Initial dataset (ChestXrayDataset or similar).
        transform (callable, optional): Image transformation pipeline.
    Returns:
        ReassignedDataset: A dataset with clusters as labels (pseudo-labels).
    """
    assert images_lists is not None

    # Initialize lists for storing image indices and their corresponding pseudo-labels
    pseudolabels = []
    image_indexes = []

    # Iterate through clusters and assign pseudo-labels
    for cluster, images in enumerate(images_lists):
        image_indexes.extend(images)
        pseudolabels.extend([cluster] * len(images))

    # If no transform is provided, use the dataset's transform (this can be modified)
    if transform is None:
        transform = dataset.transform  # Use the same transform as defined in ChestXrayDataset

    # Create and return the ReassignedDataset with pseudo-labels
    return ReassignedDataset(dataset.image_paths, pseudolabels, transform)


In [ ]:
# Define constants
NUM_EPOCHS = 10  # Total number of epochs
NUM_CLUSTERS = 1000  # Number of clusters for k-means
PCA_DIM = 256  # Dimensionality for PCA
BATCH_SIZE = 512  # Batch size for pseudo-labeled training
LEARNING_RATE = 0.05  # Learning rate for SGD
WEIGHT_DECAY = 1e-5  # Weight decay for regularization

# Instantiate the model
model = alexnet(sobel=True, bn=True, out=1000) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

# Initialize previous cluster assignments for NMI calculation
prev_cluster_assignments = None
# Start training loop with PyTorch profiler
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
             schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
             on_trace_ready=torch.profiler.tensorboard_trace_handler('./logs'),
             record_shapes=True,
             with_stack=True) as prof:

    for epoch in range(NUM_EPOCHS):
        print(f"\n### Epoch {epoch + 1}/{NUM_EPOCHS} ###")

        # Step 1: Extract features from the dataset
        with record_function("Extract Features"):
            print("Extracting features...")
            features = compute_features(dataloader, model, len(dataset), device)  # Get features from hidden layers
        
        # Step 2: Preprocess features (PCA, whitening, normalization)
        with record_function("Preprocess Features"):
            print("Preprocessing features...")
            preprocessed_features = preprocess_features(features, pca_dim=PCA_DIM)
        
        # Step 3: Perform k-means clustering
        with record_function("K-means Clustering"):
            print(f"Clustering into {NUM_CLUSTERS} clusters...")
            cluster_assignments = run_kmeans(preprocessed_features, NUM_CLUSTERS)
        
        # Step 4: Convert cluster assignments to images_lists
        images_lists = [[] for _ in range(NUM_CLUSTERS)]
        for idx, cluster_id in enumerate(cluster_assignments):
            images_lists[cluster_id].append(idx)
        
        # Step 5: Assign pseudo-labels to dataset
        with record_function("Assign Pseudo-labels"):
            print("Assigning pseudo-labels...")
            pseudo_dataset = cluster_assign(images_lists, dataset, transform=initial_transform)
            pseudo_dataloader = DataLoader(pseudo_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True)
        
        # Step 6: Train CNN with pseudo-labeled dataset
        with record_function("Train CNN"):
            print("Training CNN with pseudo-labels...")
            model.train()
            running_loss = 0.0
            for imgs, pseudo_labels in tqdm(pseudo_dataloader, desc="Training"):
                imgs, pseudo_labels = imgs.to(device), pseudo_labels.to(device)

                # Forward pass (using features from the conv5 layer)
                features = model(imgs, return_features=True)  # Get hidden features (shape: batch_size, 256, 6, 6)
                
                # Flatten the conv5 features
                features = features.view(features.size(0), -1)  # Shape: (batch_size, 256*6*6)
                
                # Compute the loss using the flattened features
                loss = criterion(features, pseudo_labels)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            avg_loss = running_loss / len(pseudo_dataloader)
            print(f"Epoch {epoch + 1} Average Loss: {avg_loss:.4f}")

        # Step 7: Evaluate clustering performance (optional)
        if prev_cluster_assignments is not None:
            with record_function("Evaluate Clustering"):
                # Calculate NMI between current and previous cluster assignments
                nmi = normalized_mutual_info_score(prev_cluster_assignments, cluster_assignments)
                print(f"Epoch {epoch + 1} NMI: {nmi:.4f}")
        
        # Save current cluster assignments for the next epoch
        prev_cluster_assignments = cluster_assignments
        prof.step()

print("Training completed.")


### Epoch 1/10 ###
Extracting features...
Computing features...


Feature Extraction: 100%|█████████████████████████████████████████████████████| 607/607 [1:45:25<00:00, 10.42s/it]


Feature extraction completed. Total time: 6325.17 seconds
Preprocessing features...
Clustering into 1000 clusters...
